# VibeVoice FastAPI Server on Kaggle 🚀

This notebook will clone `deepfak3voice` repository, install all dependencies, and launch the FastAPI server. We will use `ngrok` to create a public URL so you can test the API endpoints.

### **IMPORTANT PREREQUISITES**
1.  **Enable GPU:** In the right-hand sidebar, under **Settings > Accelerator**, select a **GPU** (e.g., T4 x2).
2.  **Enable Internet:** In the same sidebar, under **Settings > Internet**, make sure it is turned **ON**.

## Step 1: GPU Check

Let's verify that the GPU accelerator is active.

In [ ]:
!nvidia-smi

## Step 2: Clone the Repository

Now, we'll clone your project from GitHub and move into the project directory.

In [ ]:
!git clone https://github.com/iamspruce/deepfak3voice.git
%cd deepfak3voice

## Step 3: Install Dependencies

We'll install all the packages from `requirements.txt` and `pyngrok`.

In [ ]:
!pip install -q -r requirements.txt pyngrok

## Step 4: Configure Ngrok with Kaggle Secrets

Kaggle provides a secure way to store secrets like API keys.

1.  Go to the [ngrok Dashboard](https://dashboard.ngrok.com/get-started/your-authtoken) and sign up for a free account.
2.  Copy your authtoken.
3.  In this Kaggle notebook, click on **Add-ons -> Secrets** in the top menu.
4.  Create a new secret with the label `NGROK_AUTH_TOKEN` and paste your token as the value.

In [ ]:
from kaggle_secrets import UserSecretsClient
from pyngrok import ngrok

try:
    user_secrets = UserSecretsClient()
    NGROK_AUTH_TOKEN = user_secrets.get_secret("NGROK_AUTH_TOKEN")
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("Ngrok token configured successfully!")
except Exception as e:
    print(e)
    raise ValueError("Could not find Kaggle secret 'NGROK_AUTH_TOKEN'. Please ensure it's set correctly.")

## Step 5: Run the FastAPI Server! サーバーを起動する

This is the final step. We will start a public ngrok tunnel and then launch the uvicorn server. 

A public URL will be printed. **Use this URL** to access your API. The server logs will be displayed below. To stop the server, stop this cell's execution.

In [ ]:
# Start ngrok tunnel
public_url = ngrok.connect(8000)
print(f"🚀 Your VibeVoice server is live at: {public_url}")

# Run the FastAPI app
!uvicorn speak:app --host 0.0.0.0 --port 8000

## Step 6: How to Test Your API

Once the server is running, you can use `cURL` or an API client to test it.

**⚠️ Reminder:** The audio generated will be a placeholder sine wave, not actual speech, until you implement the model inference logic in `speak.py`.

### Upload a Test Audio File
First, upload a sample `.wav` or `.mp3` file to use for voice cloning. In the right-hand sidebar, go to **Data -> File -> Upload** and select your audio file. It will appear in `/kaggle/input/` but will be accessible from our current directory.

### Test with cURL
Now, create a new code cell and run the following commands. **Remember to replace `<YOUR_NGROK_URL>` with the public URL printed above and `your_audio.wav` with the name of the file you uploaded.**

**1. Health Check**
```bash
!curl -X GET <YOUR_NGROK_URL>/health
```

**2. Single Speaker TTS**
```bash
!curl -X POST -F "text=Hello Kaggle, this is a test." -F "voice_file=@your_audio.wav" <YOUR_NGROK_URL>/single-speaker -o output.wav

# You can then play the audio:
from IPython.display import Audio
Audio('output.wav')
```